# MODELLING
## Logistic Regression with Tensorflow Keras
### Fraud Detection System Development using Deep Neural Network for Reported Transactional Data

#### Import Libraries and Dataset

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

In [72]:
df = pd.read_csv('user_data_ftcr_minmax_train.csv', sep = ',')

In [73]:
df.head()

,trx_date,report_date,registereddate,birthday,is_verified,aqc_freq_prepaid_mobile,aqc_mean_prepaid_mobile_amount,aqc_freq_topup,aqc_freq_topup_within_7d,aqc_mean_topup_amount,...,job_position_KARYAWAN,job_position_LAINNYA,job_position_PEGAWAI_NS,job_position_PELAJAR,job_position_RUMAH_TANGGA,job_position_SPESIALIS,job_position_TIDAK_KERJA,job_position_WIRASWASTA,uid,is_scammer
0,0.035808,0.04750,0.261199,1.000000,1,0.097276,0.342520,0.171118,0.140187,0.003261,...,0,0,0,0,0,0,0,1,208cc2b1-7e8c-43d2-ba9f-32269abdf078,0
1,0.049782,0.06125,0.440041,0.969442,1,0.128405,0.144488,0.379822,0.205607,0.001861,...,0,0,0,0,0,0,0,1,afbeffbd-b905-4323-ac1a-3ba3e07f6951,1
2,0.452402,0.64125,0.230531,0.963121,1,0.105058,0.118701,0.067260,0.011682,0.000641,...,1,0,0,0,0,0,0,0,24b253b8-84b4-42d4-92b0-7b8537d4f066,1
3,0.027948,0.04000,0.020331,0.954634,1,0.000000,0.000000,0.012859,0.014019,0.006848,...,1,0,0,0,0,0,0,0,32414559-95f1-48fd-83ff-1ff7cc711d88,1
4,0.029694,0.04250,0.062371,0.924845,1,0.000000,0.000000,0.064293,0.060748,0.003370,...,1,0,0,0,0,0,0,0,80126b5b-7219-4266-b94b-85620d0b4498,1


In [74]:
# df = df.rename(columns={"Unnamed: 0": "id"})
df = df.drop(columns = ['uid'])
df.head()

,trx_date,report_date,registereddate,birthday,is_verified,aqc_freq_prepaid_mobile,aqc_mean_prepaid_mobile_amount,aqc_freq_topup,aqc_freq_topup_within_7d,aqc_mean_topup_amount,...,gender_None,job_position_KARYAWAN,job_position_LAINNYA,job_position_PEGAWAI_NS,job_position_PELAJAR,job_position_RUMAH_TANGGA,job_position_SPESIALIS,job_position_TIDAK_KERJA,job_position_WIRASWASTA,is_scammer
0,0.035808,0.04750,0.261199,1.000000,1,0.097276,0.342520,0.171118,0.140187,0.003261,...,0,0,0,0,0,0,0,0,1,0
1,0.049782,0.06125,0.440041,0.969442,1,0.128405,0.144488,0.379822,0.205607,0.001861,...,0,0,0,0,0,0,0,0,1,1
2,0.452402,0.64125,0.230531,0.963121,1,0.105058,0.118701,0.067260,0.011682,0.000641,...,0,1,0,0,0,0,0,0,0,1
3,0.027948,0.04000,0.020331,0.954634,1,0.000000,0.000000,0.012859,0.014019,0.006848,...,0,1,0,0,0,0,0,0,0,1
4,0.029694,0.04250,0.062371,0.924845,1,0.000000,0.000000,0.064293,0.060748,0.003370,...,0,1,0,0,0,0,0,0,0,1


In [ ]:
df.info()

#### Dataset Splitting and Shuffling

In [75]:
# Split Train and Validation Dataset
#from sklearn.model_selection import train_test_split
SPLIT_SIZE = 0.7368421

train_len = int(len(df) * SPLIT_SIZE)

shuffled_files = df.sample(frac = 1, random_state = 1)

test_set = shuffled_files[train_len:]
train_set = shuffled_files[:train_len]

y_train = train_set['is_scammer']
x_train = train_set.drop(columns=['is_scammer'])
y_test = test_set['is_scammer']
x_test = test_set.drop(columns=['is_scammer'])

In [76]:
# Convert to tensors
x_train, y_train = tf.convert_to_tensor(x_train, dtype=tf.float32), tf.convert_to_tensor(y_train, dtype=tf.float32)
x_test, y_test = tf.convert_to_tensor(x_test, dtype=tf.float32), tf.convert_to_tensor(y_test, dtype=tf.float32)

In [82]:
np.log2(2)

1.0

#### Modelling

In [109]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=64, activation='relu', input_dim=x_train.shape[1]), #Try & Error Hyperparameter
#     tf.keras.layers.Lambda(lambda x: np.log2(x+1)),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

In [110]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy', 
    metrics=[
        tf.keras.metrics.BinaryAccuracy(),
        tf.keras.metrics.Precision(), 
        tf.keras.metrics.Recall() 
#         tf.keras.metrics.FalseNegatives(),
#         tf.keras.metrics.FalsePositives(),
#         tf.keras.metrics.TruePositives(),
#         tf.keras.metrics.TrueNegatives()
    ]
)

In [111]:
epoch_count = 50

history = model.fit(
    x_train, 
    y_train, 
    epochs=epoch_count, 
    validation_data=(x_test, y_test) 
#     callbacks=[metrics]
)

Epoch 1/50
938/938 [==============================] - 2s 2ms/step - loss: 0.4854 - binary_accuracy: 0.7673 - precision_12: 0.7858 - recall_12: 0.7225 - val_loss: 0.4258 - val_binary_accuracy: 0.8057 - val_precision_12: 0.8232 - val_recall_12: 0.7769
Epoch 2/50
938/938 [==============================] - 2s 2ms/step - loss: 0.4150 - binary_accuracy: 0.8139 - precision_12: 0.8382 - recall_12: 0.7689 - val_loss: 0.3966 - val_binary_accuracy: 0.8224 - val_precision_12: 0.8758 - val_recall_12: 0.7500oss: 0.4162 - binary_accuracy: 0.8139 - precision_12: 0.8367 - recall_
Epoch 3/50
938/938 [==============================] - 2s 2ms/step - loss: 0.3934 - binary_accuracy: 0.8245 - precision_12: 0.8518 - recall_12: 0.7774 - val_loss: 0.3829 - val_binary_accuracy: 0.8282 - val_precision_12: 0.8765 - val_recall_12: 0.7627958 - binary_accuracy: 0.8221 - precision_12: 0.84
Epoch 4/50
938/938 [==============================] - 2s 2ms/step - loss: 0.3795 - binary_accuracy: 0.8331 - precision_12: 0.8601 

938/938 [==============================] - 2s 2ms/step - loss: 0.2978 - binary_accuracy: 0.8720 - precision_12: 0.9024 - recall_12: 0.8287 - val_loss: 0.3582 - val_binary_accuracy: 0.8460 - val_precision_12: 0.8845 - val_recall_12: 0.7947
Epoch 34/50
938/938 [==============================] - 2s 2ms/step - loss: 0.2971 - binary_accuracy: 0.8727 - precision_12: 0.9038 - recall_12: 0.8286 - val_loss: 0.3629 - val_binary_accuracy: 0.8447 - val_precision_12: 0.8804 - val_recall_12: 0.7966
Epoch 35/50
938/938 [==============================] - 2s 2ms/step - loss: 0.2951 - binary_accuracy: 0.8749 - precision_12: 0.9051 - recall_12: 0.8321 - val_loss: 0.3625 - val_binary_accuracy: 0.8460 - val_precision_12: 0.8747 - val_recall_12: 0.8065- binary_accuracy: 0.8765 - precision_12: 0.9067 - rec
Epoch 36/50
938/938 [==============================] - 2s 2ms/step - loss: 0.2953 - binary_accuracy: 0.8731 - precision_12: 0.9045 - recall_12: 0.8288 - val_loss: 0.3654 - val_binary_accuracy: 0.8460 - val

#### Evaluation

In [27]:
history.history.keys()

dict_keys(['loss', 'binary_accuracy', 'precision_1', 'recall_1', 'val_loss', 'val_binary_accuracy', 'val_precision_1', 'val_recall_1'])

In [112]:
for key in history.history.keys():
    print(str(key) + " : " + str(history.history[key][epoch_count-1]))

loss : 0.2795886993408203
binary_accuracy : 0.8808373212814331
precision_12 : 0.9100147485733032
recall_12 : 0.8400516510009766
val_loss : 0.3775096833705902
val_binary_accuracy : 0.8404255509376526
val_precision_12 : 0.8987035751342773
val_recall_12 : 0.7660610675811768


In [55]:
# tr_tp = history.history['true_positives'][epoch_count-1]
# tr_tn = history.history['true_negatives'][epoch_count-1]
# tr_fp = history.history['false_positives'][epoch_count-1]
# tr_fn = history.history['false_negatives'][epoch_count-1]
# val_tp = history.history['val_true_positives'][epoch_count-1]
# val_tn = history.history['val_true_negatives'][epoch_count-1]
# val_fp = history.history['val_false_positives'][epoch_count-1]
# val_fn = history.history['val_false_negatives'][epoch_count-1]

# train_acc = (tr_tp+tr_tn)/(tr_tp+tr_tn+tr_fp+tr_fn)
# val_acc = (val_tp+val_tn)/(val_tp+val_tn+val_fp+val_fn)

# tr_prec = history.history['precision'][epoch_count-1]
# tr_recall = history.history['recall'][epoch_count-1]
# val_prec = history.history['val_precision'][epoch_count-1]
# val_recall = history.history['val_recall'][epoch_count-1]

# train_f1 = (2*tr_prec*tr_recall)/(tr_prec+tr_recall)
# val_f1 = (2*val_prec*val_recall)/(val_prec+val_recall)

In [56]:
# print("Training Accuracy: " + str(train_acc))
# print("Validation Accuracy: " + str(val_acc))
# print("Training F1 score: " + str(train_f1))
# print("Validation F1 score: " + str(val_f1))

In [57]:
# import matplotlib.pyplot as plt

In [58]:
# plt.subplot(2,1,1)
# plt.plot(history.history['precision'])

# plt.subplot(2,1,2)
# plt.plot(history.history['val_precision'])

In [90]:
df_test = pd.read_csv("user_data_ftcr_minmax_test.csv", sep=',')

In [91]:
df_test = df_test.drop(columns=['uid'])
df_test.head()

,trx_date,report_date,registereddate,birthday,is_verified,aqc_freq_prepaid_mobile,aqc_mean_prepaid_mobile_amount,aqc_freq_topup,aqc_freq_topup_within_7d,aqc_mean_topup_amount,...,gender_None,job_position_KARYAWAN,job_position_LAINNYA,job_position_PEGAWAI_NS,job_position_PELAJAR,job_position_RUMAH_TANGGA,job_position_SPESIALIS,job_position_TIDAK_KERJA,job_position_WIRASWASTA,is_scammer
0,0.665502,0.93750,0.503446,0.861074,1,1.000000,0.094882,0.402572,0.584112,0.001254,...,0,0,0,0,0,1,0,0,0,0
1,0.047162,0.06750,0.052033,0.771775,1,0.000000,0.000000,0.015826,0.004673,0.007630,...,0,1,0,0,0,0,0,0,0,1
2,0.070742,0.08500,0.398001,0.770692,1,0.007782,0.004075,0.022750,0.000000,0.000633,...,0,1,0,0,0,0,0,0,0,1
3,0.007860,0.01125,0.198828,0.743068,1,0.000000,0.000000,0.010880,0.004673,0.001170,...,0,0,0,0,0,1,0,0,0,1
4,0.086463,0.12250,0.062371,0.720752,1,0.000000,0.000000,0.032641,0.028037,0.000678,...,0,0,0,0,0,1,0,0,0,1


In [115]:
test_ref_labels = df_test['is_scammer']
testing_set = df_test.drop(columns=['is_scammer'])
testing_set = tf.convert_to_tensor(testing_set, dtype=tf.float32)

In [116]:
test_pred_res = model.predict(testing_set)
tp = 0
fp = 0
tn = 0
fn = 0
for i in range (0, len(test_pred_res)):
    if test_pred_res[i] > 0.5:
        if test_ref_labels[i] == 1:
            tp += 1
        else:
            fp += 1
    else:
        if test_ref_labels[i] == 1:
            fn += 1
        else:
            tn += 1
            
print("Test Accuracy: ", (tp+tn)/(tp+tn+fp+fn))
print("Test Precision ", tp/(tp+fn))
print("Test Recall: ", tn/(tn+fp))

Test Accuracy:  0.8239962651727357
Test Precision  0.7566574839302111
Test Recall:  0.8936372269705603


In [75]:
model.input

<KerasTensor: shape=(None, 56) dtype=float32 (created by layer 'dense_18_input')>

In [ ]:
 0   trx_date                                        trx_date
 1   report_date                                     report_date
 2   registereddate                                  registereddate
 3   birthday                                        birthday
 4   is_verified                                     is_verified                                   
 5   aqc_freq_prepaid_mobile                         aqc_freq_prepaid_mobile
 6   aqc_mean_prepaid_mobile_amount                  aqc_mean_prepaid_mobile_amount
 7   aqc_freq_topup                                  aqc_freq_topup
 8   aqc_freq_topup_within_7d                        aqc_freq_topup_within_7d
 9   aqc_mean_topup_amount                           aqc_mean_topup_amount
 10  aqc_mean_topup_amount_7d                        aqc_mean_topup_amount_7d
 11  aqc_mean_topup_amount_30d                       aqc_mean_topup_amount_30d
 12  aqc_mean_topup_amount_90d                       aqc_mean_topup_amount_90d
 13  aqc_total_topup_amount_7d                       aqc_total_topup_amount_7d
 14  aqc_total_topup_amount_90d                      aqc_total_topup_amount_90d
 15  aqc_freq_x2x                                    aqc_freq_x2x
 16  aqc_freq_x2x_within_60d                         aqc_freq_x2x_within_60d
 17  aqc_freq_x2x_within_90d                         aqc_freq_x2x_within_90d
 18  aqc_mean_x2x_amount                             aqc_mean_x2x_amount
 19  aqc_mean_x2x_amount_7d                          aqc_mean_x2x_amount_7d
 20  aqc_mean_x2x_amount_30d                         aqc_mean_x2x_amount_30d
 21  aqc_mean_x2x_amount_60d                         aqc_mean_x2x_amount_60d
 22  aqc_mean_x2x_amount_90d                         aqc_mean_x2x_amount_90d
 23  aqc_total_x2x_amount                            aqc_total_x2x_amount
 24  aqc_total_x2x_amount_7d                         aqc_total_x2x_amount_7d
 25  aqc_total_x2x_amount_30d                        aqc_total_x2x_amount_30d
 26  aqc_total_x2x_amount_60d                        aqc_total_x2x_amount_60d
 27  aqc_total_x2x_amount_90d                        aqc_total_x2x_amount_90d
 28  dormancy_max_gmt_pay_diff_days                  dormancy_max_gmt_pay_diff_days
 29  dormancy_mean_gmt_pay_diff_days                 dormancy_mean_gmt_pay_diff_days
 30  dormancy_count_trx                              dormancy_count_trx
 31  kyc_total_failed                                kyc_total_failed
 32  kyc_total_revoked                               kyc_total_revoked
 33  avg_topup_weight_1                              avg_topup_weight_1
 34  avg_x2x_weight_1                                avg_x2x_weight_1
 35  avg_other_weight_1                              avg_other_weight_1
 36  centrality_outdegree_p2p                        centrality_outdegree_p2p
 37  centrality_indegree_p2p                         centrality_indegree_p2p
 38  centrality_undirected_p2p                       centrality_undirected_p2p
 39  centrality_outdegree_sendmoney                  centrality_outdegree_sendmoney
 40  account_lifetime                                account_lifetime
 41  count_trx_per_lifetime                          count_trx_per_lifetime
 42  max_gmt_pay_diff_days_per_lifetime              max_gmt_pay_diff_days_per_lifetime
 43  freq_x2x_per_lifetime                           freq_x2x_per_lifetime
 44  dormancy_max_gmt_pay_diff_days_per_count_trx    dormancy_max_gmt_pay_diff_days_per_count_trx
 45  gender_Female                                   gender_Female                                 
 46  gender_Male                                     gender_Male                                   
 47  gender_None                                     gender_None                                   
 48  job_position_KARYAWAN                           job_position_KARYAWAN                         
 49  job_position_LAINNYA                            job_position_LAINNYA                          
 50  job_position_PEGAWAI_NS                         job_position_PEGAWAI_NS                       
 51  job_position_PELAJAR                            job_position_PELAJAR                          
 52  job_position_RUMAH_TANGGA                       job_position_RUMAH_TANGGA                     
 53  job_position_SPESIALIS                          job_position_SPESIALIS                        
 54  job_position_TIDAK_KERJA                        job_position_TIDAK_KERJA                      
 55  job_position_WIRASWASTA                         job_position_WIRASWASTA                       
 56  is_scammer                                      is_scammer                                    

#### Save Model

In [67]:
# model.save("log_reg_keras_source_drop.h5")